In [1]:
import anywidget
import altair as alt
import traitlets
import pandas as pd

In [30]:
class TestWidget(anywidget.AnyWidget):
    _esm = """
    import embed from "https://cdn.jsdelivr.net/npm/vega-embed@6/+esm";
    import debounce from "https://cdn.jsdelivr.net/npm/lodash-es@4.17.21/debounce/+esm";

    export async function render({model, el}) {
        let finalize;


        const reembed = async (called_event = true) => {
            console.log(called_event);
            
            if (finalize) finalize();

             let json_spec = model.get("json_spec")

             let api;

             try {
                 api = embed(el, json_spec);
             } catch (error) {
                 console.error(error);
                 return;
             }

             finalize = api.finalize;

             model.on("change:json_spec", reembed)

        }
        
        await reembed(false)
    }
    """
    json_spec = traitlets.Dict().tag(sync=True)
    chart = traitlets.Instance(alt.TopLevelSpec)

    def __init__(self, chart, **kwargs):
        super().__init__(chart=chart, **kwargs)

    @traitlets.observe("chart")
    def _on_change_chart(self, change):
        print("New")
        new_chart = change["new"]
        self.json_spec = new_chart.to_dict()

In [39]:
df = pd.read_csv("./notebooks/avalanche/avalanches_cleaned.csv")

chart = alt.Chart(df).mark_point().encode(
    x="Elevation_feet:Q", y="Vertical_inches:Q"
)

t = TestWidget(chart=chart)
t

New


TestWidget(json_spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': …

In [42]:
c = chart.copy(deep=True)
c.data = df.sample(frac=0.99)
t.chart = c

New
